# Importing Library

In [ ]:
import numpy as np  
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# Data Set Loading

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 20
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
AUTOTUNE = tf.data.AUTOTUNE
T_PATH = "./cotton-crop-disease-detection/train"

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    T_PATH,
    shuffle = True,
    seed = 123,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names
n_classes = len(class_names)
print(f"Number of Class {n_classes} \nClass Name: {class_names}")

In [ ]:
def get_dataset_partition_tf(ds, train_split = 0.8, val_split = 0.2, shuffle = True, shuffle_size = 1000):
    assert (train_split +  val_split) == 1
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size)
    return train_ds, val_ds

In [ ]:
train_ds, val_ds = get_dataset_partition_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [ ]:
test = tf.keras.preprocessing.image_dataset_from_directory(
    "./cotton-crop-disease-detection/test",
    shuffle = True,
    seed = 123,
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

# Resize, Normalization And Data Augmentation

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.Rescaling(1.0/255),
])

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1)
])

# Model-1: MobileNetV2

In [ ]:
base = MobileNetV2(
    input_shape = INPUT_SHAPE,
    include_top = False, 
    weights = 'imagenet'
)

for layer in base.layers:
    layer.trainable = False

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
inputs = tf.keras.Input(shape = INPUT_SHAPE)
x = resize_and_rescale(inputs)
x = data_augmentation(x)
x = base(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(n_classes, activation='softmax')(x)
model_1 = Model(inputs, outputs)

model_1.compile(
              optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
)
model_1.summary()

In [ ]:
his = model_1.fit(
    train_ds,
    validation_data = val_ds,
    epochs = EPOCHS,
    callbacks=[early_stopping, lr_scheduler]
)
model_1.save("MobileNet_base.keras")

# Plots

In [ ]:
acc = his.history['accuracy']
val_acc = his.history['val_accuracy']

loss = his.history['loss']
val_loss = his.history['val_loss']
epochs_range = range(20)
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Prediction: Accuracy, Recall, Precision, F1 Score

In [ ]:
# Prediction Function 
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array, verbose = 0)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
for images_batch, labels_batch in test.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model_1.predict(images_batch, verbose= 0)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

In [ ]:
y_true = []
y_pred = []

for images, labels in test:
    preds = model_1.predict(images, verbose = 0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Precision, Recall, F1-Score
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

In [ ]:
test_losscnn, test_accuracycnn = model_1.evaluate(test, verbose=0)
print(f"Test Loss: {round(test_losscnn, 4)} \nTest Acc: {round(test_accuracycnn, 4)}")

In [ ]:
# Confusion Matrix and Heatmap
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual Labels')
plt.xlabel('Predicted Labels')
plt.show()